#라이브러리 및 데이터 임포트

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


##2021, 2022년의 날씨 데이터(미세먼지 제외)

In [ ]:
weather_2021 = pd.read_csv('/content/drive/MyDrive/데이터셋/weather/weather_2021.csv', encoding = 'cp949')
weather_2022 = pd.read_csv('/content/drive/MyDrive/데이터셋/weather/weather_2022 .csv', encoding = 'cp949')

In [ ]:
weather_2021['일시'] = pd.to_datetime(weather_2021['일시'])
weather_2022['일시'] = pd.to_datetime(weather_2022['일시'])

In [ ]:
weather_2021 = weather_2021[(weather_2021['일시'] >= '2021-05-01') & (weather_2021['일시'] < '2021-08-01')]
weather_2022 = weather_2022[(weather_2022['일시'] >= '2022-05-01') & (weather_2022['일시'] < '2022-08-01')]

In [ ]:
weather_2021['지점'].unique() == weather_2022['지점'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

##2021, 2022년의 미세먼지 데이터

In [ ]:
dust_2021 = pd.read_csv('/content/drive/MyDrive/데이터셋/weather/서울시 대기질 자료 제공_2020-2021.csv', encoding = 'cp949')
dust_2022 = pd.read_csv('/content/drive/MyDrive/데이터셋/weather/서울시 대기질 자료 제공_2022.csv', encoding = 'cp949')

In [ ]:
dust_2021['일시'] = pd.to_datetime(dust_2021['일시'])
dust_2022['일시'] = pd.to_datetime(dust_2022['일시'])

In [ ]:
dust_2021 = dust_2021[(dust_2021['일시'] >= '2021-05-01') & (dust_2021['일시'] < '2021-08-01')]
dust_2022 = dust_2022[(dust_2022['일시'] >= '2022-05-01') & (dust_2022['일시'] < '2022-08-01')]

In [ ]:
dust_2021 = dust_2021[dust_2021['구분']=='평균']
dust_2022 = dust_2022[dust_2022['구분']=='평균']

#관측소와 클러스터 매칭: 가까운 거리 기준
클러스터의 좌표는, 클러스터 내 대여소의 평균 좌표

##클러스터 내 평균 좌표

In [ ]:
clusters_for_live = pd.read_csv('/content/drive/MyDrive/데이터셋/clustering/version_2/clusters_for_live')
clusters_for_work = pd.read_csv('/content/drive/MyDrive/데이터셋/clustering/version_2/clusters_for_work')

In [ ]:
clusters_total = pd.concat([clusters_for_live, clusters_for_work]).sort_values('cluster')

In [ ]:
clusters_locations = pd.DataFrame(clusters_total.groupby('cluster')['latitude', 'longitude'].mean()).reset_index()
clusters_locations

<ipython-input-83-4b2b6c3d6404>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clusters_locations = pd.DataFrame(clusters_total.groupby('cluster')['latitude', 'longitude'].mean()).reset_index()


,cluster,latitude,longitude
0,0,37.517115,126.869460
1,1,37.558498,127.075855
2,2,37.478357,127.124176
3,3,37.646911,127.066212
4,4,37.581627,126.909230
...,...,...,...
395,395,37.536528,126.973933
396,396,37.475195,126.901123
397,397,37.503831,127.113303
398,398,37.508565,127.041870


##클러스터별 최근접 관측소 찾기

In [ ]:
weather_locations = pd.read_csv('/content/drive/MyDrive/데이터셋/weather/META_관측지점정보 .csv', encoding = 'cp949')
weather_locations = weather_locations[['지점', '위도', '경도']]
weather_locations = weather_locations[weather_locations['지점'].isin(weather_2021['지점'].unique())==True]
weather_locations = weather_locations.drop_duplicates(subset = '지점', keep = 'last')
weather_locations

In [ ]:
weather_locations['지점'].unique() == weather_2022['지점'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [ ]:
near_list = []
for i in clusters_locations['cluster']:
  distance_list = []
  cluster_location = clusters_locations[clusters_locations['cluster']==i]
  for j in weather_locations['지점']:
    distance = np.sqrt((cluster_location['latitude'].values[0] - weather_locations[weather_locations['지점']==j]['위도'].values[0])**2 + (cluster_location['longitude'].values[0] - weather_locations[weather_locations['지점']==j]['경도'].values[0])**2)
    distance_list.append(distance)
  near_list.append(weather_locations['지점'].unique()[distance_list.index(min(distance_list))])

In [ ]:
clusters_locations['near_weather'] = near_list
clusters_locations

,cluster,latitude,longitude,near_weather
0,0,37.517115,126.869460,405
1,1,37.558498,127.075855,413
2,2,37.478357,127.124176,403
3,3,37.646911,127.066212,406
4,4,37.581627,126.909230,416
...,...,...,...,...
395,395,37.536528,126.973933,415
396,396,37.475195,126.901123,417
397,397,37.503831,127.113303,403
398,398,37.508565,127.041870,400


#Train Test set에 가까운 관측소 라벨 붙이기

In [ ]:
weather_2021 = weather_2021.drop(['지점명', '습도(%)'], axis = 1).reset_index(drop=True)
weather_2021.columns = ['지점', '일시', '기온', '풍속', '강수량']
weather_2021

,지점,일시,기온,풍속,강수량
0,116,2021-05-01 00:00:00,5.8,3.1,0.0
1,116,2021-05-01 01:00:00,5.9,2.6,0.0
2,116,2021-05-01 02:00:00,5.6,1.9,0.0
3,116,2021-05-01 03:00:00,6.0,0.7,0.0
4,116,2021-05-01 04:00:00,5.3,1.4,0.0
...,...,...,...,...,...
60843,889,2021-07-31 19:00:00,31.1,2.3,0.0
60844,889,2021-07-31 20:00:00,28.7,0.2,0.0
60845,889,2021-07-31 21:00:00,29.1,0.6,0.0
60846,889,2021-07-31 22:00:00,29.0,1.1,0.0


In [ ]:
weather_2022 = weather_2022.drop(['지점명', '풍향(deg)', '습도(%)'], axis = 1).reset_index(drop=True)
weather_2022.columns = ['지점', '일시', '기온', '풍속', '강수량']
weather_2022

,지점,일시,기온,풍속,강수량
0,116,2022-05-01 00:00:00,6.0,4.4,0.0
1,116,2022-05-01 01:00:00,6.3,3.1,0.0
2,116,2022-05-01 02:00:00,5.8,3.3,0.0
3,116,2022-05-01 03:00:00,5.7,2.6,0.0
4,116,2022-05-01 04:00:00,6.9,4.5,0.0
...,...,...,...,...,...
61735,889,2022-07-31 19:00:00,26.9,1.4,0.5
61736,889,2022-07-31 20:00:00,26.5,1.2,4.0
61737,889,2022-07-31 21:00:00,26.0,1.1,9.0
61738,889,2022-07-31 22:00:00,25.8,0.3,14.0


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/데이터셋/clustering/version_2/train_clustered.csv')
train['시간'] = pd.to_datetime(train['시간'])
train = pd.merge(train, clusters_locations, how = 'left', left_on='cluster', right_on = 'cluster')
train = train[['cluster', '시간', '순대여량', 'near_weather']]
train

,cluster,시간,순대여량,near_weather
0,0,2021-05-01 00:00:00,0.0,405
1,0,2021-05-01 01:00:00,1.0,405
2,0,2021-05-01 02:00:00,-2.0,405
3,0,2021-05-01 03:00:00,2.0,405
4,0,2021-05-01 04:00:00,0.0,405
...,...,...,...,...
883195,399,2021-07-31 19:00:00,-7.0,406
883196,399,2021-07-31 20:00:00,-5.0,406
883197,399,2021-07-31 21:00:00,0.0,406
883198,399,2021-07-31 22:00:00,-6.0,406


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/데이터셋/clustering/version_2/test_clustered.csv')
test['시간'] = pd.to_datetime(test['시간'])
test = pd.merge(test, clusters_locations, how = 'left', left_on='cluster', right_on = 'cluster')
test = test[['cluster', '시간', '순대여량', 'near_weather']]
test

,cluster,시간,순대여량,near_weather
0,0,2022-05-01 00:00:00,-7.0,405
1,0,2022-05-01 01:00:00,8.0,405
2,0,2022-05-01 02:00:00,1.0,405
3,0,2022-05-01 03:00:00,0.0,405
4,0,2022-05-01 04:00:00,2.0,405
...,...,...,...,...
883195,399,2022-07-31 19:00:00,0.0,406
883196,399,2022-07-31 20:00:00,0.0,406
883197,399,2022-07-31 21:00:00,0.0,406
883198,399,2022-07-31 22:00:00,-1.0,406


#Train Test에 날씨 데이터 붙이기(미세먼지 제외)
<br>결측치는 ffill: 직전 시간, 해당 관측소의 날씨값이 됨
<br>결측치 중, 직전 행이 다른 클러스터인 경우는 없는 것 확인

In [ ]:
train = pd.merge(train, weather_2021, how='left', left_on = ['시간', 'near_weather'], right_on = ['일시', '지점'])
train = train.fillna(method = 'ffill')
train

,cluster,시간,순대여량,near_weather,지점,일시,기온,풍속,강수량
0,0,2021-05-01 00:00:00,0.0,405,405.0,2021-05-01 00:00:00,11.3,1.3,0.0
1,0,2021-05-01 01:00:00,1.0,405,405.0,2021-05-01 01:00:00,11.2,1.6,0.0
2,0,2021-05-01 02:00:00,-2.0,405,405.0,2021-05-01 02:00:00,10.8,1.5,0.0
3,0,2021-05-01 03:00:00,2.0,405,405.0,2021-05-01 03:00:00,10.9,0.3,0.0
4,0,2021-05-01 04:00:00,0.0,405,405.0,2021-05-01 04:00:00,10.9,1.3,0.0
...,...,...,...,...,...,...,...,...,...
883195,399,2021-07-31 19:00:00,-7.0,406,406.0,2021-07-31 19:00:00,31.2,2.0,0.0
883196,399,2021-07-31 20:00:00,-5.0,406,406.0,2021-07-31 20:00:00,29.7,1.3,0.0
883197,399,2021-07-31 21:00:00,0.0,406,406.0,2021-07-31 21:00:00,28.8,1.4,0.0
883198,399,2021-07-31 22:00:00,-6.0,406,406.0,2021-07-31 22:00:00,28.6,2.0,0.0


In [ ]:
train.isna().sum()

cluster         0
시간              0
순대여량            0
near_weather    0
지점              0
일시              0
기온              0
풍속              0
강수량             0
dtype: int64

In [ ]:
test = pd.merge(test, weather_2022, how='left', left_on = ['시간', 'near_weather'], right_on = ['일시', '지점'])
test = test.fillna(method = 'ffill')
test

,cluster,시간,순대여량,near_weather,지점,일시,기온,풍속,강수량
0,0,2022-05-01 00:00:00,-7.0,405,405.0,2022-05-01 00:00:00,11.9,1.2,0.0
1,0,2022-05-01 01:00:00,8.0,405,405.0,2022-05-01 01:00:00,11.5,0.8,0.0
2,0,2022-05-01 02:00:00,1.0,405,405.0,2022-05-01 02:00:00,11.1,0.9,0.0
3,0,2022-05-01 03:00:00,0.0,405,405.0,2022-05-01 03:00:00,10.0,0.4,0.0
4,0,2022-05-01 04:00:00,2.0,405,405.0,2022-05-01 04:00:00,9.8,1.1,0.0
...,...,...,...,...,...,...,...,...,...
883195,399,2022-07-31 19:00:00,0.0,406,406.0,2022-07-31 19:00:00,26.7,0.6,0.5
883196,399,2022-07-31 20:00:00,0.0,406,406.0,2022-07-31 20:00:00,26.5,0.7,0.5
883197,399,2022-07-31 21:00:00,0.0,406,406.0,2022-07-31 21:00:00,26.1,1.1,17.5
883198,399,2022-07-31 22:00:00,-1.0,406,406.0,2022-07-31 22:00:00,25.9,0.8,16.5


In [ ]:
test.isna().sum()

cluster         0
시간              0
순대여량            0
near_weather    0
지점              0
일시              0
기온              0
풍속              0
강수량             0
dtype: int64

In [ ]:
train = train.drop(['near_weather', '지점', '일시'], axis = 1)
test = test.drop(['near_weather', '지점', '일시'], axis = 1)

In [ ]:
train

,cluster,시간,순대여량,기온,풍속,강수량
0,0,2021-05-01 00:00:00,0.0,11.3,1.3,0.0
1,0,2021-05-01 01:00:00,1.0,11.2,1.6,0.0
2,0,2021-05-01 02:00:00,-2.0,10.8,1.5,0.0
3,0,2021-05-01 03:00:00,2.0,10.9,0.3,0.0
4,0,2021-05-01 04:00:00,0.0,10.9,1.3,0.0
...,...,...,...,...,...,...
883195,399,2021-07-31 19:00:00,-7.0,31.2,2.0,0.0
883196,399,2021-07-31 20:00:00,-5.0,29.7,1.3,0.0
883197,399,2021-07-31 21:00:00,0.0,28.8,1.4,0.0
883198,399,2021-07-31 22:00:00,-6.0,28.6,2.0,0.0


In [ ]:
test

,cluster,시간,순대여량,기온,풍속,강수량
0,0,2022-05-01 00:00:00,-7.0,11.9,1.2,0.0
1,0,2022-05-01 01:00:00,8.0,11.5,0.8,0.0
2,0,2022-05-01 02:00:00,1.0,11.1,0.9,0.0
3,0,2022-05-01 03:00:00,0.0,10.0,0.4,0.0
4,0,2022-05-01 04:00:00,2.0,9.8,1.1,0.0
...,...,...,...,...,...,...
883195,399,2022-07-31 19:00:00,0.0,26.7,0.6,0.5
883196,399,2022-07-31 20:00:00,0.0,26.5,0.7,0.5
883197,399,2022-07-31 21:00:00,0.0,26.1,1.1,17.5
883198,399,2022-07-31 22:00:00,-1.0,25.9,0.8,16.5


#Train Test set에 미세먼지 데이터 붙이기
<br> 해당 시간대 서울시 평균값으로

In [ ]:
dust_2021 = dust_2021.drop('구분', axis = 1)
dust_2021.columns = ['시간', '미세먼지', '초미세먼지']
dust_2021 = dust_2021.reset_index(drop=True)
dust_2021

,시간,미세먼지,초미세먼지
0,2021-07-31 23:00:00,21.0,15.0
1,2021-07-31 22:00:00,18.0,12.0
2,2021-07-31 21:00:00,18.0,12.0
3,2021-07-31 20:00:00,23.0,15.0
4,2021-07-31 19:00:00,26.0,17.0
...,...,...,...
2203,2021-05-01 04:00:00,36.0,23.0
2204,2021-05-01 03:00:00,33.0,20.0
2205,2021-05-01 02:00:00,29.0,16.0
2206,2021-05-01 01:00:00,31.0,17.0


In [ ]:
dust_2022 = dust_2022.drop('구분', axis = 1)
dust_2022.columns = ['시간', '미세먼지', '초미세먼지']
dust_2022 = dust_2022.reset_index(drop=True)
dust_2022

,시간,미세먼지,초미세먼지
0,2022-07-31 23:00:00,6.0,4.0
1,2022-07-31 22:00:00,8.0,5.0
2,2022-07-31 21:00:00,11.0,7.0
3,2022-07-31 20:00:00,12.0,8.0
4,2022-07-31 19:00:00,11.0,7.0
...,...,...,...
2203,2022-05-01 04:00:00,31.0,17.0
2204,2022-05-01 03:00:00,34.0,19.0
2205,2022-05-01 02:00:00,40.0,25.0
2206,2022-05-01 01:00:00,47.0,31.0


In [ ]:
train_weather = pd.merge(train, dust_2021, how = 'left', left_on = '시간', right_on = '시간')
train_weather

,cluster,시간,순대여량,기온,풍속,강수량,미세먼지,초미세먼지
0,0,2021-05-01 00:00:00,0.0,11.3,1.3,0.0,31.0,16.0
1,0,2021-05-01 01:00:00,1.0,11.2,1.6,0.0,31.0,17.0
2,0,2021-05-01 02:00:00,-2.0,10.8,1.5,0.0,29.0,16.0
3,0,2021-05-01 03:00:00,2.0,10.9,0.3,0.0,33.0,20.0
4,0,2021-05-01 04:00:00,0.0,10.9,1.3,0.0,36.0,23.0
...,...,...,...,...,...,...,...,...
883195,399,2021-07-31 19:00:00,-7.0,31.2,2.0,0.0,26.0,17.0
883196,399,2021-07-31 20:00:00,-5.0,29.7,1.3,0.0,23.0,15.0
883197,399,2021-07-31 21:00:00,0.0,28.8,1.4,0.0,18.0,12.0
883198,399,2021-07-31 22:00:00,-6.0,28.6,2.0,0.0,18.0,12.0


In [ ]:
train_weather.isna().sum()

cluster    0
시간         0
순대여량       0
기온         0
풍속         0
강수량        0
미세먼지       0
초미세먼지      0
dtype: int64

In [ ]:
test_weather = pd.merge(test, dust_2022, how = 'left', left_on = '시간', right_on = '시간')
test_weather

,cluster,시간,순대여량,기온,풍속,강수량,미세먼지,초미세먼지
0,0,2022-05-01 00:00:00,-7.0,11.9,1.2,0.0,52.0,35.0
1,0,2022-05-01 01:00:00,8.0,11.5,0.8,0.0,47.0,31.0
2,0,2022-05-01 02:00:00,1.0,11.1,0.9,0.0,40.0,25.0
3,0,2022-05-01 03:00:00,0.0,10.0,0.4,0.0,34.0,19.0
4,0,2022-05-01 04:00:00,2.0,9.8,1.1,0.0,31.0,17.0
...,...,...,...,...,...,...,...,...
883195,399,2022-07-31 19:00:00,0.0,26.7,0.6,0.5,11.0,7.0
883196,399,2022-07-31 20:00:00,0.0,26.5,0.7,0.5,12.0,8.0
883197,399,2022-07-31 21:00:00,0.0,26.1,1.1,17.5,11.0,7.0
883198,399,2022-07-31 22:00:00,-1.0,25.9,0.8,16.5,8.0,5.0


In [ ]:
test_weather.isna().sum()

cluster    0
시간         0
순대여량       0
기온         0
풍속         0
강수량        0
미세먼지       0
초미세먼지      0
dtype: int64

#저장

In [ ]:
#train_weather.to_csv('/content/drive/MyDrive/데이터셋/weather/train_clustered_weather.csv', index = False)
#test_weather.to_csv('/content/drive/MyDrive/데이터셋/weather/test_clustered_weather.csv', index = False)

In [ ]:
train_weather

,cluster,시간,순대여량,기온,풍속,강수량,미세먼지,초미세먼지
0,0,2021-05-01 00:00:00,0.0,11.3,1.3,0.0,31.0,16.0
1,0,2021-05-01 01:00:00,1.0,11.2,1.6,0.0,31.0,17.0
2,0,2021-05-01 02:00:00,-2.0,10.8,1.5,0.0,29.0,16.0
3,0,2021-05-01 03:00:00,2.0,10.9,0.3,0.0,33.0,20.0
4,0,2021-05-01 04:00:00,0.0,10.9,1.3,0.0,36.0,23.0
...,...,...,...,...,...,...,...,...
883195,399,2021-07-31 19:00:00,-7.0,31.2,2.0,0.0,26.0,17.0
883196,399,2021-07-31 20:00:00,-5.0,29.7,1.3,0.0,23.0,15.0
883197,399,2021-07-31 21:00:00,0.0,28.8,1.4,0.0,18.0,12.0
883198,399,2021-07-31 22:00:00,-6.0,28.6,2.0,0.0,18.0,12.0


In [ ]:
test_weather

,cluster,시간,순대여량,기온,풍속,강수량,미세먼지,초미세먼지
0,0,2022-05-01 00:00:00,-7.0,11.9,1.2,0.0,52.0,35.0
1,0,2022-05-01 01:00:00,8.0,11.5,0.8,0.0,47.0,31.0
2,0,2022-05-01 02:00:00,1.0,11.1,0.9,0.0,40.0,25.0
3,0,2022-05-01 03:00:00,0.0,10.0,0.4,0.0,34.0,19.0
4,0,2022-05-01 04:00:00,2.0,9.8,1.1,0.0,31.0,17.0
...,...,...,...,...,...,...,...,...
883195,399,2022-07-31 19:00:00,0.0,26.7,0.6,0.5,11.0,7.0
883196,399,2022-07-31 20:00:00,0.0,26.5,0.7,0.5,12.0,8.0
883197,399,2022-07-31 21:00:00,0.0,26.1,1.1,17.5,11.0,7.0
883198,399,2022-07-31 22:00:00,-1.0,25.9,0.8,16.5,8.0,5.0
